In [1]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

# img = read_image("test/assets/encode_jpeg/grace_hopper_517x606.jpg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

import torch 
img = torch.rand(3, 224,224)
# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)
prediction = model(batch).squeeze(0).softmax(0)

In [4]:
model.fc 

Linear(in_features=2048, out_features=1000, bias=True)

In [13]:
def foward_resnet(resnet, x):
    # See note [TorchScript super()]
    x = resnet.conv1(x)
    x = resnet.bn1(x)
    x = resnet.relu(x)
    x = resnet.maxpool(x)

    x = resnet.layer1(x)
    x = resnet.layer2(x)
    x = resnet.layer3(x)
    x = resnet.layer4(x)

    x = resnet.avgpool(x)
    x = torch.flatten(x, 1)
    x = resnet.fc(x)
    return x

def foward_resnet(resnet, x):
    # See note [TorchScript super()]
    x = resnet.conv1(x)
    x = resnet.bn1(x)
    x = resnet.relu(x)
    x = resnet.maxpool(x)

    for m in model.layer1:
        x = forward_bottleneck(m, x)
    for m in model.layer2:
        x = forward_bottleneck(m, x)
    for m in model.layer3:
        x = forward_bottleneck(m, x)
    for m in model.layer4:
        x= forward_bottleneck(m, x)

    x = resnet.avgpool(x)
    x = torch.flatten(x, 1)
    x = resnet.fc(x)
    return x

def forward_bottleneck(model, x):
    assert model.__class__.__name__ == "Bottleneck"
    print(model.__class__.__name__ )
    identity = x

    out = model.conv1(x)
    out = model.bn1(out)
    out = model.relu(out)
    out = model.conv2(out)
    out = model.bn2(out)
    out = model.relu(out)
    out = model.conv3(out)
    out = model.bn3(out)
    if model.downsample is not None:
        identity = model.downsample(x)
    out += identity
    out = model.relu(out)

    return out

x = torch.rand(1, 3, 224,224)
foward_resnet(model, x).size()

Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck
Bottleneck


torch.Size([1, 1000])

AttributeError: 'super' object has no attribute '__getattr__'